In [56]:
from notebook_utils import resolve_paths_from_parent_directory
resolve_paths_from_parent_directory()
# auto reload notebook deps
%reload_ext autoreload
%autoreload 2

In [57]:
import json
import pandas as pd

DATASETS = {
    "CentralParkNYC": {
        "input_location": "../data/tweet_objects/CentralParkNYC/2021-01-27-2021-02-06.json",
        "output_location": "../data/labeled_datasets/CentralParkNYC-2021-01-27-2021-02-06.json",
        "spike_start": "Feb 1, 2021, 17:00",
        "spike_end": "Feb 3, 2021, 04:00",
    },
    "united": {
        "input_location": "../data/tweet_objects/united/2020-12-05-2020-12-15.json",
        "output_location": "../data/labeled_datasets/united-2020-12-05-2020-12-15.json",
        "spike_start": "Dec 12, 2020, 05:00",
        "spike_end": "Dec 13, 2020, 06:00",
    },
    "nationalGridUS": {
        "input_location": "../data/tweet_objects/nationalGridUS/2020-10-01-2020-10-12.json",
        "output_location": "../data/labeled_datasets/nationalGridUS-2020-10-01-2020-10-12.json",
        "spike_start": "Oct 7, 2020, 17:00",
        "spike_end": "Oct 9, 2020, 04:00",
    },
    "soccer_gareth_bale": {
        "input_location": "../data/tweet_objects/soccer_gareth_bale/2021-01-01-2021-04-01.json",
        "output_location": "../data/labeled_datasets/soccer_gareth_bale.json",
        "spike_start": "Oct 7, 2020, 17:00",
        "spike_end": "Oct 9, 2020, 04:00",
    },
    "soccer_raphael_varane": {
        "input_location": "../data/tweet_objects/soccer_raphael_varane/2021-01-01-2021-04-01.json",
        "output_location": "../data/labeled_datasets/soccer_raphael_varane.json",
        "spike_start": "Oct 7, 2020, 17:00",
        "spike_end": "Oct 9, 2020, 04:00",
    },
    "celebrity_martin_garrix": {
        "input_location": "../data/tweet_objects/celebrity_martin_garrix/2021-01-01-2021-04-01.json",
        "output_location": "../data/labeled_datasets/celebrity_martin_garrix.json",
        "spike_start": "Oct 7, 2020, 17:00",
        "spike_end": "Oct 9, 2020, 04:00",
    },
    "celebrity_jk_rowling": {
        "input_location": "../data/tweet_objects/celebrity_jk_rowling/2021-01-01-2021-04-01.json",
        "output_location": "../data/labeled_datasets/celebrity_jk_rowling.json",
        "spike_start": "Oct 7, 2020, 17:00",
        "spike_end": "Oct 9, 2020, 04:00",
    },
    "voterfraud2020": {
        "input_location": "../data/voterfraud2020/processed/df_sample.json",
        "output_location": "../data/labeled_datasets/voterfraud2020_sample.json",
        "spike_start": "Oct 7, 2020, 17:00",
        "spike_end": "Oct 9, 2020, 04:00",
    },
    "celebrity_jerry_seinfeld": {
        "input_location": "../data/tweet_objects/celebrity_jerry_seinfeld/2021-01-01-2021-04-01.json",
        "output_location": "../data/labeled_datasets/celebrity_jerry_seinfeld.json",
        "spike_start": "Oct 7, 2020, 17:00",
        "spike_end": "Oct 9, 2020, 04:00",
    },
    "celebrity_akon": {
        "input_location": "../data/tweet_objects/celebrity_akon/2021-01-01-2021-04-01.json",
        "output_location": "../data/labeled_datasets/celebrity_akon.json",
        "spike_start": "Oct 7, 2020, 17:00",
        "spike_end": "Oct 9, 2020, 04:00",
    },
    "soccer_alan_shearer": {
        "input_location": "../data/tweet_objects/soccer_alan_shearer/2021-01-01-2021-04-01.json",
        "output_location": "../data/labeled_datasets/soccer_alan_shearer.json",
        "spike_start": "Oct 7, 2020, 17:00",
        "spike_end": "Oct 9, 2020, 04:00",
    },
}

selected_dataset = DATASETS["soccer_alan_shearer"]
labeled_spike_start = pd.to_datetime(selected_dataset["spike_start"]).tz_localize("UTC")
labeled_spike_end = pd.to_datetime(selected_dataset["spike_end"]).tz_localize("UTC")

In [58]:
from utils.dataset import create_tweet_df

if "voterfraud2020" in selected_dataset["input_location"]:
    df_tweets = pd.read_json(selected_dataset["input_location"])
    df_tweets["created_at"] = pd.to_datetime(df_tweets["created_at"]).dt.tz_localize("UTC")
    df_tweets.sort_values("created_at", inplace=True)
else:
    with open(selected_dataset["input_location"], "r") as f:
        raw_dataset = json.load(f)

    df_tweets = create_tweet_df(raw_dataset["tweets"])

# lower entities
df_tweets["hashtags"] = df_tweets.hashtags.apply(lambda xs: [x.lower() for x in xs])
df_tweets["mentions"] = df_tweets.mentions.apply(lambda xs: [x.lower() for x in xs])

# Set retweet count 0 for retweets
df_tweets["retweet_count"] = df_tweets.apply(
    lambda x: x.retweet_count if x.retweeted is None else 0, 
    axis=1
)

# Label tweets
df_tweets["is_anomaly"] = df_tweets.created_at.apply(
    lambda x: 0 if (x < labeled_spike_start or x > labeled_spike_end) else 1
)

df_tweets.head()

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,retweet_count,quote_count,reply_count,like_count,replied_to,retweeted,quoted,is_anomaly
49888,1344796226953371648,@Hyto1992 @DeAtHaToMiC69 @2911_leon @Coster_ra...,2021-01-01 00:03:00+00:00,[],"[hyto1992, deathatomic69, 2911_leon, lfc_lione...",3167011317,1203755312383217669,0,0,1,0,1344791195017424901,NaN,NaN,0
49887,1344796398076780545,@Hyto1992 @DeAtHaToMiC69 @2911_leon @Coster_ra...,2021-01-01 00:03:41+00:00,[],"[hyto1992, deathatomic69, 2911_leon, lfc_lione...",1203755312383217669,1203755312383217669,0,0,0,0,1344796226953371648,NaN,NaN,0
49886,1344796442192449536,"RT @premierleague: Roll up, roll up... 🥁\n\n@a...",2021-01-01 00:03:51+00:00,[],"[premierleague, alanshearer]",None,929316358508048385,0,0,0,0,NaN,1344691089333219328,NaN,0
49885,1344796509632667657,"RT @premierleague: Roll up, roll up... 🥁\n\n@a...",2021-01-01 00:04:07+00:00,[],"[premierleague, alanshearer]",None,1086176370789662720,0,0,0,0,NaN,1344691089333219328,NaN,0
49884,1344796688398102529,"RT @premierleague: Roll up, roll up... 🥁\n\n@a...",2021-01-01 00:04:50+00:00,[],"[premierleague, alanshearer]",None,1318835761,0,0,0,0,NaN,1344691089333219328,NaN,0


In [59]:
## Create timeseries from tweets
from utils.dataset import count_array_column

df_top_hashtags = count_array_column(df_tweets["hashtags"])
df_top_mentions = count_array_column(df_tweets["mentions"])

df_top_hashtags[:20]

,value,count,pct
5,nufc,2705,0.054220
1421,brinew,1377,0.027601
217,ashleyout,662,0.013269
125,onthisday,586,0.011746
63,pl,534,0.010704
57,bruceout,530,0.010624
127,facup,422,0.008459
436,otd,417,0.008359
1057,liveve,350,0.007016
767,footballfocus,252,0.005051


In [60]:
def count_col_occurrence(df_col, value):
    return df_col.apply(lambda values: value in values).sum()

time_bucket_size = "300Min"

df_timeseries = df_tweets.groupby(df_tweets.created_at.dt.ceil(time_bucket_size)).agg(
    total_count=('id', 'count'), 
    is_anomaly=('is_anomaly', lambda x: x.any()),
    retweet_count=('retweeted', lambda x: pd.notna(x).sum()),
    quote_count=('quoted', lambda x: pd.notna(x).sum()),
    # replied_to_count=('replied_to', lambda x: pd.notna(x).sum()),
    top1_hashtag_count=(
        'hashtags', 
        lambda x: count_col_occurrence(x, df_top_hashtags.iloc[0].value)
    ),
    top2_hashtag_count=(
        'hashtags', 
        lambda x: count_col_occurrence(x, df_top_hashtags.iloc[1].value)
    ),
    top3_hashtag_count=(
        'hashtags', 
        lambda x: count_col_occurrence(x, df_top_hashtags.iloc[2].value)
    ),
    top1_mention_count=(
        'mentions', 
        lambda x: count_col_occurrence(x, df_top_mentions.iloc[0].value)
    ),
    top2_mention_count=(
        'mentions', 
        lambda x: count_col_occurrence(x, df_top_mentions.iloc[1].value)
    ),
    top3_mention_count=(
        'mentions', 
        lambda x: count_col_occurrence(x, df_top_mentions.iloc[2].value)
    )
)
df_timeseries.head()

,total_count,is_anomaly,retweet_count,quote_count,top1_hashtag_count,top2_hashtag_count,top3_hashtag_count,top1_mention_count,top2_mention_count,top3_mention_count
created_at,,,,,,,,,,
2021-01-01 03:00:00+00:00,94,False,35,0,0,0,0,91,0,72
2021-01-01 08:00:00+00:00,75,False,32,0,0,0,0,74,0,73
2021-01-01 13:00:00+00:00,87,False,28,1,3,0,0,84,2,69
2021-01-01 18:00:00+00:00,48,False,15,0,2,0,0,42,0,30
2021-01-01 23:00:00+00:00,67,False,16,5,0,0,0,58,6,14


In [61]:
from merlion.utils import TimeSeries
from merlion.models.anomaly.forecast_based.prophet import ProphetDetector, ProphetDetectorConfig
from merlion.plot import plot_anoms_plotly
from merlion.post_process.threshold import Threshold

def convert_to_merlion(df, column):
    df = df.copy()
    df = df[column]
    df.index = df.index.tz_convert(None)
    return TimeSeries.from_pd(df)

def detect_anomalies(
    df_timeseries, 
    column,
    model=ProphetDetector(ProphetDetectorConfig(
        threshold=Threshold(alm_threshold=0.5, abs_score=False),
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False,
        add_seasonality=False,
        uncertainty_samples=500,
    )),
    plot=True
):
    train_data = convert_to_merlion(df_timeseries, column)
    anomaly_score = model.train(train_data=train_data, anomaly_labels=None)
    scores = model.get_anomaly_score(train_data)
    df_scores = scores.to_pd()
    labels_train = model.get_anomaly_label(train_data)

    fig = model.plot_anomaly_plotly(
        time_series=train_data,
        plot_forecast=True,
        plot_forecast_uncertainty=True
    )
    plot_anoms_plotly(fig, anomaly_labels=labels_train)
    fig.show()

    df_labels = labels_train.to_pd()
    df_labels.resample(time_bucket_size).fillna("bfill")

    def set_anom_score(created_at):
        lookup = created_at.tz_convert(None).ceil(time_bucket_size)
        if lookup in df_labels.index:
            return df_labels.loc[lookup].anom_score
        else:
            return 0    

    ## Label df_tweets with anomaly
    df_tweets["merlion_anomaly_" + column] = df_tweets.created_at.apply(
        set_anom_score
    )
    return df_scores

#anomaly_scores = detect_anomalies(
#    df_timeseries, 
#    "total_count"
#)

In [62]:
import plotly.express as px


px.line(
    df_timeseries.total_count
)

In [63]:
from merlion.transform.moving_average import MovingAverage
from merlion.post_process.threshold import Threshold

prophet_config = ProphetDetectorConfig(
    threshold=Threshold(alm_threshold=0.5, abs_score=False),
    # transform=MovingAverage(n_steps=3),
    yearly_seasonality=False,
    weekly_seasonality=False,
    daily_seasonality=False,
    add_seasonality=False,  
    uncertainty_samples=500
)

#anomaly_scores = detect_anomalies(
#    df_timeseries, 
#    "total_count",
#    model=ProphetDetector(prophet_config)
#)

In [64]:
for feat in ["total_count", "top1_hashtag_count", "top2_hashtag_count", "top3_hashtag_count"]:
    detect_anomalies(
        df_timeseries, 
        feat,
        model=ProphetDetector(prophet_config)
    )

Initial log joint probability = -4.99525
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      69       718.137   3.25451e-05       101.608   3.855e-07       0.001      121  LS failed, Hessian reset 
      99        718.14   8.43841e-07       96.3686      0.8712      0.8712      157   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     122        718.14   2.15112e-07         96.82   2.009e-09       0.001      225  LS failed, Hessian reset 
     134        718.14   2.01874e-08       81.9515     0.03631           1      242   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


Initial log joint probability = -3.54224
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      26       858.988    0.00103321       101.407   1.006e-05       0.001       64  LS failed, Hessian reset 
      93       859.096   3.14898e-08       96.1514      0.2922           1      149   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


Initial log joint probability = -2.97568
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      32       958.475   1.11095e-07       98.5901       0.637       0.637       59   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


Initial log joint probability = -2.97113
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      23       959.491   2.29297e-08       100.219      0.5586      0.5586       54   
Optimization terminated normally: 
  Convergence detected: relative gradient magnitude is below tolerance


In [65]:
# Export 
df_tweets.to_json(
    selected_dataset["output_location"],
    orient="records"
)
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49889 entries, 49888 to 0
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype              
---  ------                              --------------  -----              
 0   id                                  49889 non-null  object             
 1   text                                49889 non-null  object             
 2   created_at                          49889 non-null  datetime64[ns, UTC]
 3   hashtags                            49889 non-null  object             
 4   mentions                            49889 non-null  object             
 5   in_reply_to_user_id                 24907 non-null  object             
 6   user_id                             49889 non-null  object             
 7   retweet_count                       49889 non-null  int64              
 8   quote_count                         49889 non-null  int64              
 9   reply_count                         498